# Coordinate Transforms

A geoscientist is often left to collate data from multiple different sources. If they are lucky these will all be in the same coordinate system, but  this is not alwayts the case. 

Tools like [arcgis](https://www.arcgis.com/index.html), or [QGis](https://www.qgis.org/) exist thart can transform these objects using a Graphical User Interface but say we have Millions of points, large meshes, or objects not supported by these packages we need a way to be able to homogenise the coordinate systems.

## Import Libraries

Different solutions and libraires exist to carry out this task and I am not aiming to list them exhaustivly. Here I will use the [pyproj](https://pypi.org/project/pyproj/) library because I am familir with it.

We will be calling the `transform` function of `pyproj`. This signature of this function is `inProjection`, `outProjection`, `xvals`, `yvals` (`zvals`, `radians` are optional). The in and out projections are `Proj` objects internal to `pyproj`.To create a `Proj` object you can use [EPSG](http://epsg.io/) to find the code

In [1]:
from pyproj import Proj, transform

### Input Projection

Here the input projection is [WGS84 Psuedo Mercator](http://epsg.io/3857). This is projected coordinate system used for rendering maps in Google Maps, OpenStreetMap, etc.

In [2]:
inProj = Proj(init='epsg:3857')

### Output projection
And we want to project this to longitude and latitude [WGS84](http://epsg.io/4326)

In [3]:
outProj = Proj(init='epsg:4326')

### Test Coordinates
So lets declare some input coordinates

In [31]:
x = 12897531.863054074
y = -3756814.7353761178

### Transformed Coordinates

In [32]:
long, lat = transform(inProj,outProj,x, y)

In [33]:
lat

-31.950500000000016

In [34]:
long

115.86050000000002

### Convert multiple

The transform can receive multiple input coordinates in the same coordinate system!!! 

In [85]:
x = [12897531.863054074, 15428959.347591272, 17034676.21058977, 16343582.547846964]
y = [-3756814.7353761178, -4154168.819822751, -3182290.6656223023, -2185452.8280846006]

In [87]:
longs, lats = transform(inProj, outProj, x, y)

In [88]:
longs

[115.86050000000002, 138.6007, 153.0251, 146.81689999999995]

In [89]:
lats

[-31.950500000000016, -34.928500000000014, -27.469799999999996, -19.259]

## View Results

We are going to use a library called ipyleaflet to view the results

In [91]:
# Import
from ipyleaflet import Marker, Map, basemaps

# Declare the centre
center = (-25.2744, 133.7751)

# Create the map
m = Map(center=center, zoom=4, basemap=basemaps.Esri.WorldImagery)
# Add the marker
for i in range(len(longs)):
    marker = Marker(location=(lats[i], longs[i]), draggable=False)
    m.add_layer(marker);

# Show the map
m

Map(basemap={'url': 'http://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/…

## Mesh Transform

In [113]:
import os

In [116]:
filename = os.path.join('..', 'Data', 'mesh.obj')
filename

'..\\Data\\mesh.obj'

In [212]:
mesh = {'v' : [], 'f' : []}
with open(filename, 'r') as f:
    line = f.readline()
    while line:
        line = line.split()
        mesh[line[0]].append(line[1:])
        line = f.readline()
mesh['v'] = numpy.array(mesh['v']).astype(numpy.float).tolist()

In [153]:
xs, ys, zs = numpy.transpose(mesh['v'])

In [171]:
xmin = numpy.min(xs)
ymin = numpy.min(ys)
zmin = numpy.min(zs)

xmax = numpy.max(xs)
ymax = numpy.max(ys)
zmax = numpy.max(zs)

In [205]:
ipv.figure()
ipv.scatter(xs, ys, zs)
ipv.xlim(xmin, xmax)
ipv.ylim(ymin, ymax)
ipv.zlim(zmin, zmax)
ipv.show()

## convert

In [206]:
longs, lats = transform(inProj, outProj, xs, ys)

## Export

In [210]:
verts = numpy.array([longs, lats, zs]).astype(str).T

In [213]:
with open('test.obj', 'w') as f:
    for vert in verts:
        f.write('v ' + "\t".join(vert))
    for tri in mesh['f']:
        f.write('f ' + "\t".join(tri))